In [ ]:
import sqlite3
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
import matplotlib.cm as cm

from ipyleaflet import Map, CircleMarker, LayerGroup, WidgetControl, basemaps
from ipywidgets import Button, Layout, HTML, Checkbox, VBox, Accordion
from IPython.display import display, HTML as DispHTML
import matplotlib.colors as mcolors

display(DispHTML("""
<style>
  .leaflet-control.widget-control {
    overflow-x: hidden !important;
  }
</style>
"""))

INITIAL_CENTER = (37.8, -96.0)
INITIAL_ZOOM   = 4
MIN_RADIUS     = 5
MAX_RADIUS     = 15
CMAP           = cm.get_cmap("YlOrRd")

def load_data(n=3000):
    conn = sqlite3.connect('mapdata/wildfiredataset/FPA_FOD_20170508.sqlite')
    
    df = pd.read_sql_query(
        "SELECT FIRE_YEAR, DISCOVERY_DOY, CONT_DOY, FIRE_SIZE, LATITUDE, LONGITUDE "
        "FROM Fires ORDER BY RANDOM() LIMIT ?", conn,
        params=(n,)
    )
    conn.close()
    df["DAY_TO_CONT"] = df["CONT_DOY"] - df["DISCOVERY_DOY"]
    return df.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

m = Map(
    center=INITIAL_CENTER,
    zoom=INITIAL_ZOOM,
    min_zoom=3,
    max_zoom=10,
    basemap=basemaps.CartoDB.Positron,
    scroll_wheel_zoom=True
)

counter = HTML()
m.add_control(WidgetControl(widget=counter, position='topleft'))

year_layers     = {}
year_counts     = {}
year_checkboxes = {}

def update_counter():
    total = sum(
        year_counts[yr] for yr, cb in year_checkboxes.items() if cb.value
    )
    counter.value = (
        f"<div style='background:white; padding:6px 10px; "
        f"border-radius:4px; box-shadow:0 1px 3px rgba(0,0,0,0.2); "
        f"font-size:14px;'><b>Total Fires:</b> {total}</div>"
    )

def plot_markers(df):
    for grp in year_layers.values():
        try: m.remove_layer(grp)
        except: pass
    year_layers.clear()
    year_counts.clear()

    coords = np.vstack([df["LATITUDE"], df["LONGITUDE"]])
    if len(df) > 10:
        try:
            kde  = gaussian_kde(coords)
            dens = kde(coords)
            vmin, vmax = dens.min(), dens.max()
        except:
            vmin, vmax = 0,1
    else:
        vmin, vmax = 0,1

    for year in sorted(df["FIRE_YEAR"].unique()):
        subset = df[df["FIRE_YEAR"] == year]
        cnt    = len(subset)
        year_counts[str(year)] = cnt

        grp = LayerGroup(name=str(year))
        pts = np.vstack([subset["LATITUDE"], subset["LONGITUDE"]])
        N   = pts.shape[1]

        if N >= 3 and len(df) > 10:
            try:
                dvals = kde(pts)
                norm  = np.clip((dvals - vmin)/(vmax - vmin), 0, 1)
            except:
                norm = np.full(N, 0.5)
        else:
            norm = np.full(N, 0.2)

        sizes = subset["FIRE_SIZE"].fillna(0)
        if sizes.max() != sizes.min():
            snorm = (sizes - sizes.min())/(sizes.max() - sizes.min())
        else:
            snorm = np.zeros_like(sizes)
        radii = MIN_RADIUS + snorm*(MAX_RADIUS - MIN_RADIUS)

        for lat, lon, r, d in zip(
            subset["LATITUDE"], subset["LONGITUDE"], radii, norm
        ):
            grp.add_layer(CircleMarker(
                location=(lat, lon),
                radius=int(r),
                fill=True,
                fill_color=mcolors.to_hex(CMAP(d)),
                fill_opacity=0.9,
                stroke=False
            ))

        year_layers[str(year)] = grp
        if year_checkboxes[str(year)].value:
            m.add_layer(grp)

def make_checkboxes(years):
    boxes = []
    for year in years:
        cb = Checkbox(
            description=str(year),
            value=True,
            style={'description_width': 'initial'},
            layout=Layout(
                width='auto',
                padding='0px 0px 0px 6px'
            )
        )
        def on_toggle(change, yr=str(year)):
            if change['new']:
                m.add_layer(year_layers[yr])
            else:
                m.remove_layer(year_layers[yr])
            update_counter()
        cb.observe(on_toggle, names='value')
        year_checkboxes[str(year)] = cb
        boxes.append(cb)

    return VBox(
        boxes,
        layout=Layout(
            max_height='120px',
            overflow_y='auto',
            overflow_x='hidden',
            align_items='flex-start'
        )
    )

df0 = load_data()
years = sorted(df0["FIRE_YEAR"].unique())

boxes = make_checkboxes(years)
acc = Accordion(children=[boxes], layout=Layout(width='auto'))
acc.set_title(0, 'Years')
acc.selected_index = None

checkbox_ctrl_list = [WidgetControl(widget=acc, position='topright')]
m.add_control(checkbox_ctrl_list[0])

plot_markers(df0)
update_counter()

btn = Button(description="New Random Sample", layout=Layout(width="180px"))
def on_random(_):
    m.remove_control(checkbox_ctrl_list[0])

    df_new = load_data()
    new_years = sorted(df_new["FIRE_YEAR"].unique())
    new_boxes = make_checkboxes(new_years)
    new_acc = Accordion(children=[new_boxes], layout=Layout(width='auto'))
    new_acc.set_title(0, 'Years')
    new_acc.selected_index = None

    new_ctrl = WidgetControl(widget=new_acc, position='topright')
    m.add_control(new_ctrl)
    checkbox_ctrl_list[0] = new_ctrl

    plot_markers(df_new)
    update_counter()

btn.on_click(on_random)
m.add_control(WidgetControl(widget=btn, position='bottomleft'))

reset_btn = Button(description="Reset View", layout=Layout(width="180px"))
reset_btn.on_click(lambda _: (
    setattr(m, 'center', INITIAL_CENTER),
    setattr(m, 'zoom', INITIAL_ZOOM)
))
m.add_control(WidgetControl(widget=reset_btn, position='bottomleft'))

high = mcolors.to_hex(CMAP(1.0))
low  = mcolors.to_hex(CMAP(0.0))
legend_html = HTML(value=f"""
<div style="
    background: white;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 0 1px 5px rgba(0,0,0,0.2);
    font-family: Arial, sans-serif;
    font-size: 12px;
    min-width: 140px;
">
  <h4 style="text-align:center;">Fire Density</h4>
  <div style="display:flex; align-items:center; margin:4px 0;">
    <div style="
      width:14px; height:14px; border-radius:50%;
      background:{high}; border:1px solid rgba(0,0,0,0.3);
      margin-right:8px;
    "></div> High (Clustered)
  </div>
  <div style="display:flex; align-items:center; margin:4px 0;">
    <div style="
      width:14px; height:14px; border-radius:50%;
      background:{low}; border:1px solid rgba(0,0,0,0.3);
      margin-right:8px;
    "></div> Low (Isolated)
  </div>
</div>
""")
m.add_control(WidgetControl(widget=legend_html, position='bottomright'))

display(m)

/var/folders/fp/jc___qwn0c7bdmx39c3_r96w0000gn/T/ipykernel_59891/1236762422.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  CMAP           = cm.get_cmap("YlOrRd")


Map(center=[37.8, -96.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [8]:
from ipywidgets.embed import embed_minimal_html
embed_minimal_html('wildfire_density_total_years_map.html', views=[m], title='Wildfire Density Map')